In [ ]:
import sympy as sp 
import numpy as np
import matplotlib.pyplot as plt
import cayley_schreier as cs

In [ ]:
#define symbols
t1, t2 = sp.symbols('t1 t2', real=True)
k = sp.symbols('k', real=True)
symbols = [k, t1, t2]

In [ ]:
params = {
    "N_pbc": 400, #number of k-points in the Brillouin zone
    "N_obc": 100, #number of sites in the open direction
    "num_bands": 4, #number of bands
    "t1": 1.0, #hopping parameter model a and b
    "t2": 2.0, #hopping parameter model a and b
}

In [ ]:
#create the Bloch Hamiltonians for models a and b
Hk_a = cs.create_triangular_hamiltonian(symbols, model = 'a')
Hk_b = cs.create_triangular_hamiltonian(symbols, model = 'b')

In [ ]:
#substitute hopping parameters
Hk_a_subs = Hk_a.subs({t1:params["t1"], t2:params["t2"]})
Hk_b_subs = Hk_b.subs({t1:params["t1"], t2:params["t2"]})

#lambdify the Hamiltonians for the spectrum calculation
Hk_a_f = sp.lambdify(k, Hk_a_subs, modules='numpy')
Hk_b_f = sp.lambdify(k, Hk_b_subs, modules='numpy')

In [ ]:
#define BZ
Ks = np.linspace(0, 2*np.pi, params["N_pbc"], endpoint=False)

In [ ]:
#calculate pbc spectrum
eigenvals_a, eigenvecs_a = cs.spectrum(Hk_a_f, [Ks], params["num_bands"])
eigenvals_b, eigenvecs_b = cs.spectrum(Hk_b_f, [Ks], params["num_bands"])

In [ ]:
#make the bands continuous
eigenvecs_a, eigenvals_a = cs.continuous_bands_1d(vals = eigenvals_a, vecs = eigenvecs_a)
eigenvecs_b, eigenvals_b = cs.continuous_bands_1d(vals = eigenvals_b, vecs = eigenvecs_b)

#compute the partial polarization invariant
invariant_a = cs.compute_partial_polarization(eigenvecs_a[:,:,0])
invariant_b = cs.compute_partial_polarization(eigenvecs_b[:,:,0])

print("Partial polarization of model a:", invariant_a)
print("Partial polarization of model b:", invariant_b)

In [ ]:
#calculate obc Hamiltonians
Ha_obc = np.array(cs.get_surface_hamiltonian(Hk_a_subs, k, params["N_obc"], params["num_bands"])).astype(np.complex128)
Hb_obc = np.array(cs.get_surface_hamiltonian(Hk_b_subs, k, params["N_obc"], params["num_bands"])).astype(np.complex128)

#calculate obc spectrum
valsa_obc = np.linalg.eigvalsh(Ha_obc)
valsb_obc = np.linalg.eigvalsh(Hb_obc)

In [ ]:
# combine the two figures into a single one
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))

for n in range(params["num_bands"]):
    ax1.plot(Ks, eigenvals_a[:, n], c="black")
    ax1.plot(Ks, eigenvals_b[:, n], c="green", linestyle="-")

ax1.set_xlabel(r"$\mathsf{k}$", fontsize=14, labelpad=0)
ax1.set_ylabel(r"$\mathsf{\epsilon}$", fontsize=14, labelpad=-5)
ax1.set_xticks([0, np.pi, 2*np.pi],
               labels=[r"$\mathsf{0}$", r"$\mathsf{\pi}$", r"$\mathsf{2\pi}$"],
               fontsize=14)
ax1.set_yticks([-3, 0, 3],
               [r"$\mathsf{-3}$", r"$\mathsf{0}$", r"$\mathsf{3}$"],
               fontsize=14)

ax2.plot(valsa_obc, 'o', markersize=2, color="black", label="model 1")
ax2.plot(valsb_obc, 'x', markersize=2, color="green", label="model 2")

ax2.set_xlabel(r"$\mathsf{state~index}$", fontsize=14)
ax2.set_xticks([0, 200, 400],
               [r"$\mathsf{0}$", r"$\mathsf{200}$", r"$\mathsf{400}$"],
               fontsize=14)
ax2.set_yticks([-3, 0, 3],
               [r"$\mathsf{-3}$", r"$\mathsf{0}$", r"$\mathsf{3}$"],
               fontsize=14)

plt.legend(fontsize=12, loc="upper left", markerscale=3)
plt.tight_layout()
#plt.savefig("../figures/triangular_spectrum.svg", dpi = 1000, bbox_inches = "tight")
plt.show()

In [ ]:
fr = params["num_bands"]*params["N_obc"]//2 - 3
to = params["num_bands"]*params["N_obc"]//2 + 3
mid = params["num_bands"]*params["N_obc"]//2


fig, ax = plt.subplots(figsize=(2, 2))
indices = np.arange(len(valsa_obc))

ax.scatter(indices[fr:to], valsa_obc[fr:to], color='black', marker='o')
ax.scatter(indices[fr:to], valsb_obc[fr:to], c='green', marker='x')
ax.set_yticks([-0.5, 0., 0.5], [r"$\mathsf{-0.5}$", r"$\mathsf{0.0}$", r"$\mathsf{0.5}$"], fontsize=18)
ax.set_xticks([mid], [fr"$\mathsf{{{mid}}}$"], fontsize=18)
#plt.savefig("../figures/triangular_spectrum_zoom.svg", dpi = 1000, bbox_inches = "tight")
plt.show()
